In [3]:
import sys

import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
def carrega_dados(file_name, separador=',', drop_cols=None):
    data = pd.read_csv(f"../data/{file_name}", sep=separador)
    print("Reorganizando os dados...")
    data = data.sample(frac=1)

    try:
        data = data if not drop_cols else data.drop(drop_cols, axis=1)
    except KeyError:
        print(f"Colunas {drop_cols} não encontradas no DataFrame.")
        sys.exit(1)

    print("Dados carregados")
    return data

def encoding_cols(data, cols):
    print("Transformando colunas categóricas em numéricas...")

    if not data.columns.isin(cols).any():
        print(f"Colunas {cols} não encontradas no DataFrame.")
        raise KeyError(f"Colunas {cols} não encontradas no DataFrame.")
    

    label_encoder = LabelEncoder()

    for col in cols:
        if data[col].dtype == 'object':
            data[col] = label_encoder.fit_transform(data[col])
        else:
            print(f"A coluna {col} não é categórica, não será transformada.")
            raise ValueError(f"A coluna {col} não é categórica, não será transformada.")
        
    print("Colunas categóricas transformadas em numéricas")
    
    return data

def standardize_data(X_train, X_test):
    print("Normalizandos os dados com z-score")
    scaler = StandardScaler()

    if not isinstance(X_train, pd.DataFrame) or not isinstance(X_test, pd.DataFrame):
        raise ValueError("X_train e X_test devem ser DataFrames do pandas.")
    
    if X_train.shape[1] != X_test.shape[1]:
        raise ValueError("X_train e X_test devem ter o mesmo número de colunas.")
    
    if X_train.isnull().values.any() or X_test.isnull().values.any():
        raise ValueError("X_train e X_test não podem conter valores nulos.")

    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    print("Dados normalizados com z-score")

    scaler_filename = 'scaler.pkl'
    with open(f"../artifacts/{scaler_filename}", 'wb') as file:
        pickle.dump(scaler, file)
    return X_train, X_test

def standardize_data_from_file(data, scaler_filename):
    print("Normalizando os dados com z-score")
    scaler = pickle.load(open(f"../artifacts/{scaler_filename}", 'rb'))

    if not isinstance(data, pd.DataFrame):
        raise ValueError("data devem ser DataFrames do pandas.")
    
    if data.isnull().values.any():
        raise ValueError("data não pode conter valores nulos.")

    data = scaler.transform(data)
    print("Dados normalizados com z-score")

    return data

def save_model(model, filename):
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    print("Modelo salvo")

def load_model(filename):
    with open(filename, 'rb') as file:
        model = pickle.load(file)
    return model

def get_x_y(data, y_label, x_cols=None):
    print("Preparando amostras de treino e validação")
    try:
        X = data.drop(y_label, axis=1)
        X = X if not x_cols else X[x_cols]
        y = data[y_label]
    except KeyError:
        print(f"Colunas {y_label} ou {x_cols} não encontradas no DataFrame.")
        sys.exit(1)

    print("Amostras de treino e validação preparadas")

    return X, y


In [4]:
data = carrega_dados("heart_disease_dataset.csv", drop_cols=["Gender"])
data_train, data_test = train_test_split(data, test_size=0.1)

Reorganizando os dados...
Dados carregados


In [5]:
X, y = get_x_y(data_train, "Heart Disease")

Preparando amostras de treino e validação
Amostras de treino e validação preparadas


In [6]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [ ]:
X_train = encoding_cols(X_train, X_train.select_dtypes(include=['object']).columns.tolist())
X_val = encoding_cols(X_val, X_val.select_dtypes(include=['object']).columns.tolist())

C, X_val = standardize_data(X_train, X_val)

Transformando colunas categóricas em numéricas...
Colunas categóricas transformadas em numéricas
Transformando colunas categóricas em numéricas...
Colunas categóricas transformadas em numéricas
Normalizandos os dados com z-score
Dados normalizados com z-score


In [8]:
svm_clf = SVC(kernel="linear")
svm_clf.fit(X_train, y_train)

SVC(kernel='linear')

In [9]:
pred = svm_clf.predict(X_val)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.90      0.90      0.90       124
           1       0.78      0.77      0.77        56

    accuracy                           0.86       180
   macro avg       0.84      0.84      0.84       180
weighted avg       0.86      0.86      0.86       180



In [11]:
pred

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 1])